In [42]:
import sqlite3

import requests
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool


# def get_engine_for_chinook_db():
#     """Pull sql file, populate in-memory database, and create engine."""
#     url = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql"
#     response = requests.get(url)
#     sql_script = response.text

#     connection = sqlite3.connect(":memory:", check_same_thread=False)
#     connection.executescript(sql_script)
#     return create_engine(
#         "sqlite://",
#         creator=lambda: connection,
#         poolclass=StaticPool,
#         connect_args={"check_same_thread": False},
#     )


# engine = get_engine_for_chinook_db()

# from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///rental.db")

# db = SQLDatabase(engine)

In [51]:
import getpass
import os
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore")

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [44]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [45]:
toolkit.get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7fd5bf9607f0>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7fd5bf9607f0>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x7fd5bf9607f0>),
 QuerySQLCheckerTool(description='Use this tool to double check

In [46]:
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)

In [47]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

/home/nathalierocelle/sql-agent/venv/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


['dialect', 'top_k']


In [48]:
system_message = prompt_template.format(dialect="SQLite", top_k=5)

In [52]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    llm, toolkit.get_tools(), state_modifier=system_message
)

example_query = "What is the most rented film?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most rented film?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_7KpUZjfqMnVOfPWoKaphtOpS)
 Call ID: call_7KpUZjfqMnVOfPWoKaphtOpS
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

actor, address, category, city, country, customer, film, film_actor, film_category, film_text, inventory, language, payment, rental, staff, store
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_YS8YhhtrJmd4op2BBR2EmC1i)
 Call ID: call_YS8YhhtrJmd4op2BBR2EmC1i
  Args:
    table_names: film
  sql_db_schema (call_tdBpkM5bS3bYpSWuG6qS74LW)
 Call ID: call_tdBpkM5bS3bYpSWuG6qS74LW
  Args:
    table_names: rental
================================= Tool Message =================================
Name: sql_db_schema


CREATE 

In [50]:
example_query = "Who are the top 3 best selling actors?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Who are the top 3 best selling actors?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_rKuZmNb3RpCsjHWZPXdFxkov)
 Call ID: call_rKuZmNb3RpCsjHWZPXdFxkov
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

actor, address, category, city, country, customer, film, film_actor, film_category, film_text, inventory, language, payment, rental, staff, store
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_puPsIPi8xREUcfcToTVFaRZe)
 Call ID: call_puPsIPi8xREUcfcToTVFaRZe
  Args:
    table_names: actor
  sql_db_schema (call_cKGL6xqUkjYq0cEj9QHEeEuh)
 Call ID: call_cKGL6xqUkjYq0cEj9QHEeEuh
  Args:
    table_names: film_actor
  sql_db_schema (call_NG8Ye12jm82PU3uIQY9I1gok)
 Call ID: call_NG8Ye12jm82PU3uIQY9I1gok
  Args:


/home/nathalierocelle/sql-agent/venv/lib/python3.10/site-packages/langchain_community/utilities/sql_database.py:317: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  metadata_table_names = [tbl.name for tbl in self._metadata.sorted_tables]
/home/nathalierocelle/sql-agent/venv/lib/python3.10/site-packages/langchain_community/utilities/sql_database.py:329: SAWarning: Cannot correctly sort tables; there are unresolvable cycles between tables "staff, store", which is usually caused by mutually dependent foreign key constraints.  Foreign key constraints involving these tables will not be considered; this warning may raise an error in a future release.
  for tbl in self._metadata.sorted_tables
/home/nathalierocelle/sql-agent/venv/lib/python3.10/

================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_qqBh5baDzsbkLMywmTiBuSyk)
 Call ID: call_qqBh5baDzsbkLMywmTiBuSyk
  Args:
    query: SELECT actor_id, COUNT(film_id) AS film_count FROM film_actor GROUP BY actor_id ORDER BY film_count DESC LIMIT 3;
================================= Tool Message =================================
Name: sql_db_query

[(107, 42), (102, 41), (198, 40)]
================================== Ai Message ==================================
Tool Calls:
  sql_db_query (call_6sIacI9SVvrk4JlbkTKK71ug)
 Call ID: call_6sIacI9SVvrk4JlbkTKK71ug
  Args:
    query: SELECT first_name, last_name FROM actor WHERE actor_id = 107;
  sql_db_query (call_UbenYHaK6xGvWaC4iWjemlyT)
 Call ID: call_UbenYHaK6xGvWaC4iWjemlyT
  Args:
    query: SELECT first_name, last_name FROM actor WHERE actor_id = 102;
  sql_db_query (call_B6eskceXVcr6X5am5GAdKTcn)
 Call ID: call_B6eskceXVcr6X5am5GAdKTcn
  Args:
    query: SELECT first_name